In [17]:
from utility.utils import *
from dataset.dataset import *
from dataset.transform import *
from model.loss import *
from model.unet import UNet256_3x3
from training.classifier import NucleiClassifier
from model.eval import *
from dataset.submission import *

import torch
from torch.autograd import Variable as V
import torch.nn.functional as F
import torch.optim as optim

from skimage.morphology import label
from collections import OrderedDict
import matplotlib.pyplot as plt
import cv2
import shutil
import os

### Create Random (Stratified) Train and Validation Set - RUN AS NEEDED

In [18]:
classes = pd.read_csv('classes.csv')
train_dirs, valid_dirs = get_stratified_valid_dirs(classes, seed=17)

In [19]:
main_path = '../data/ds_bowl_2018/'
full_path = '../data/ds_bowl_2018/full_data/'

In [20]:
create_validation_dirs(main_path, full_path, None, train_dirs, valid_dirs)

Copied 606 training and 64 validation data


In [21]:
# test distribution
classes[~classes.is_train].groupby(['foreground', 'background']).count()

filename  is_train
foreground background                    
purple     purple             1         1
           white              3         3
           yellow             8         8
white      black             53        53

In [22]:
# distributions seem similar to test
classes[classes.filename.isin([f + '.png' for f in valid_dirs])].groupby(['foreground', 'background']).count()

filename  is_train
foreground background                    
purple     purple             2         2
           white              4         4
white      black             58        58

### Training Data Prep

In multiclass UNET we will not be outputing a single channel like we did for binary classification background or mask but rather this time we will have a 3 channel output. Each channel will correspond to probability distributions for background, mask and overlap. We will be using multiclass BCE Loss.

Output Channels:

- Background
- Foreground
- Overlap


If there are 3 unique pixel intensities

- 30 : Background
- 110 : Nuclei
- 215 : Overlap Boundary

If there are 2 unique pixel intensities

- 30 : Background
- 215 : Nuclei

w_background, w_nuclei, w_overlap = (1.1551767249306626, 7.544967099214484, 557.753645718466)

In [23]:
train_path = '../data/ds_bowl_2018/train/'
valid_path = '../data/ds_bowl_2018/valid/'
dummy_path = '../data/ds_bowl_2018/dummy/'

In [24]:
train_dirs = list_directory(train_path)
valid_dirs = list_directory(valid_path)
dummy_dirs = list_directory(dummy_path)

In [25]:
list_directory(train_dirs[0])

['../data/ds_bowl_2018/train/e4537e7893e631f3ba6ae5b1023e24b233c78249a31c2f5e561f6c4cad88fcf6/mclass_one_mask.png/',
 '../data/ds_bowl_2018/train/e4537e7893e631f3ba6ae5b1023e24b233c78249a31c2f5e561f6c4cad88fcf6/masks/',
 '../data/ds_bowl_2018/train/e4537e7893e631f3ba6ae5b1023e24b233c78249a31c2f5e561f6c4cad88fcf6/images/']

In [26]:
len(train_dirs), len(valid_dirs), len(dummy_dirs)

(606, 64, 2)

In [27]:
def get_3d_mask(mask):
    if len(np.unique(mask)) == 2:
        back_channel = (mask == 30)*1 # background
        nuclei_channel = (mask == 215)*1 # nuclei
        overlap_channel = np.zeros_like(mask) # overlap - missing so all 0s
    else:
        back_channel = (mask == 30)*1 # background
        nuclei_channel = (mask == 110)*1 # nuclei
        overlap_channel = (mask == 215)*1 # overlap
    # stack depth-wise
    multiclass_mask = np.dstack([back_channel, nuclei_channel, overlap_channel])
    return multiclass_mask

In [28]:
def train_augment(image, mask, index, sz = 256):           
    # get 3d mask
    mask3d = get_3d_mask(mask).astype(np.uint8) 
    # resize image, mask for Unet256
    image = fix_resize_transform(image, sz, sz)
    mask3d = fix_resize_transform(mask3d, sz, sz) 
    # get dimensions right for pytorch
    image = (image.transpose((2,0,1))) / 255 
    mask3d = (mask3d.transpose((2,0,1)))
    return image.astype(np.float64), mask3d.astype(np.float64), index

def valid_augment(image, mask, index, sz = 256):        
    # get 3d mask
    mask3d = get_3d_mask(mask).astype(np.uint8) 
    # resize image, mask for Unet256
    image = fix_resize_transform(image, sz, sz)
    mask3d = fix_resize_transform(mask3d, sz, sz) 
    # get dimensions right for pytorch
    image = (image.transpose((2,0,1))) / 255 
    mask3d = (mask3d.transpose((2,0,1)))
    return image.astype(np.float64), mask3d.astype(np.float64), index


def test_augment(image, mask, index, sz = 256):
    # resize image for Unet256
    image = fix_resize_transform(image, sz, sz)
    # normalize pixel intensities
    image = (image.transpose((2,0,1))) / 255 
    return image.astype(np.float64), index

In [29]:
# # create dummyloader
dummy_ds = NucleiDataset('../data/ds_bowl_2018/dummy/', transform=train_augment, mode='train', mask_file='/mclass_one_mask.png')
dummy_dl = DataLoader(dummy_ds, batch_size=1, shuffle=False)
# create trainloader
trn_ds = NucleiDataset('../data/ds_bowl_2018/train/', transform=train_augment, mode='train', mask_file='/mclass_one_mask.png')
trn_dl = DataLoader(trn_ds, batch_size=16, shuffle=True)
# create validloader.png
val_ds = NucleiDataset('../data/ds_bowl_2018/valid/', transform=valid_augment, mode='valid', mask_file='/mclass_one_mask.png')
val_dl = DataLoader(val_ds, batch_size=16, shuffle=False)
# create testloader
test_ds = NucleiDataset('../data/ds_bowl_2018/test/', transform=test_augment, mode='test')
test_dl = DataLoader(test_ds, batch_size=1, shuffle=False)

In [30]:
class Weighted_BCELoss2d(nn.Module):
    """
    Weights for a single sample which is repeated along the batch
    Inputs:
        weight: weigth tensor for a single sample
    """
    def __init__(self, weight=None):
        super(Weighted_BCELoss2d, self).__init__()
        self.weight = weight
        
    def forward(self, logits, targets):
        # contruct bce loss
        batch_size = targets.size(0)
        weights = self.weight.repeat(batch_size)
        bce_loss = nn.BCELoss(weights)
        # calc loss
        probs        = F.softmax(logits, dim=1)
        probs_flat   = probs.view (-1)
        targets_flat = targets.view(-1)
        return bce_loss(probs_flat, targets_flat)

In [31]:
w_background, w_nuclei, w_overlap = (1.155, 7.544, 557.753)
weights = torch.cat([torch.ones(256*256)*w_background, 
                     torch.ones(256*256)*w_nuclei, torch.ones(256*256)*w_overlap]).double().cuda()

In [43]:
# training definition
net = UNet256_3x3(in_shape=(3, 256, 256), num_classes=3).double().cuda()
#optimizer = optim.Adam(net.parameters(), lr=0.1)
optimizer = optim.RMSprop(net.parameters(), lr=0.1)
# weighted BCELoss
crit = Weighted_BCELoss2d(weights)

In [44]:
# init classifier
classifier = NucleiClassifier(net, 1000)

In [ ]:
# train for 10 epochs
for i in range(10):
    classifier.train(train_loader=trn_dl, valid_loader=val_dl, optimizer=optimizer,crit=crit, epochs=10, threshold=0.5)
    print(f'saving at {(i+1)*10}')
    classifier.save_model(f'./models/mclass_{(i+1)*10}', None)

Epoch: 21
Training : [2.5705],Validation : [3.0883]
Epoch: 22
Training : [2.9707],Validation : [2.2333]
Epoch: 23
Training : [2.6349],Validation : [2.0003]
Epoch: 24
Training : [2.7624],Validation : [2.1028]
Epoch: 25
Training : [2.6743],Validation : [1.8726]
Epoch: 26
Training : [2.6271],Validation : [1.9951]
Epoch: 27
Training : [2.6622],Validation : [2.1075]
Epoch: 28
Training : [2.6003],Validation : [3.1161]
Epoch: 29
Training : [2.573],Validation : [3.0118]
Epoch: 30
Training : [2.6178],Validation : [2.4611]
saving at 10
Epoch: 31
Training : [2.6216],Validation : [3.418]
Epoch: 32
Training : [2.4947],Validation : [2.8201]
Epoch: 33
Training : [2.4566],Validation : [1.9171]
Epoch: 34
Training : [2.526],Validation : [1.9593]
Epoch: 35
Training : [2.5589],Validation : [2.2711]
Epoch: 36
Training : [2.438],Validation : [1.8514]
Epoch: 37
Training : [2.5089],Validation : [1.7949]
Epoch: 38
Training : [2.4422],Validation : [1.747]
Epoch: 39
Training : [2.491],Validation : [1.8385]
Epoch

### Evaluate by Eye

In [ ]:
def show_side_to_side(im1, im2, figsize=(13, 13)):
    plt.figure(figsize=figsize)
    plt.subplot(1,2,1)
    plt.imshow(im1)
    plt.subplot(1,2,2)
    plt.imshow(im2)
    plt.show()
    plt.close()

In [ ]:
val_ds = NucleiDataset('../data/ds_bowl_2018/valid/', transform=valid_augment, mode='valid', mask_file='/mclass_one_mask.png')
val_dl = DataLoader(val_ds, batch_size=1, shuffle=False)

In [ ]:
it = iter(val_dl)

In [ ]:
img, mask, _ = next(it)

net = classifier.net.eval()

out = net(V(img).cuda())

prob, idx = torch.max(out, 1)

show_with_sz(idx.cpu().data.numpy()[0])

In [ ]:
show_with_sz(img[0].numpy().transpose((1,2,0)))